In [1]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import pandas_ta as ta

from datetime import datetime, date

In [2]:
def grab_symbols():
    # Grab S&P Symbols from Wikipedia or local HTML File
    # wiki_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    # wiki_url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies#S&P_500_component_stocks'
    tickers = pd.read_html('./tickers.html')[0]
    tickers = tickers.Symbol.to_list()
    tickers = [i.replace('.','-') for i in tickers]
    return tickers 

print('Grab symbols function defined...')

Grab symbols function defined...


In [3]:
def yahoo_prices(tickers, current_day):
    # Return a DataFrame with dates, symbols and prices
    if not isinstance(tickers,list):
        return None

    years_back = 5
    end_dt = datetime.strptime(current_day, '%Y-%m-%d').date()
    start_dt = date(end_dt.year - years_back, end_dt.month, end_dt.day)

    data = yf.download(tickers, start=start_dt, end=end_dt)
    data = data.loc[(slice(None)),(slice(None),slice(None))].copy()
    data = data.stack()
    data = data.reset_index()
    data.rename(columns={'level_1': 'Symbol'}, inplace=True)
    data.set_index('Date', inplace=True)
    return data

print('Yahoo_prices function defined...')    

Yahoo_prices function defined...


In [106]:
def multi_asset_list(all_assets):
    multi_asset = []
    tickers = all_assets.Symbol.unique().tolist()
    start_date = all_assets.index.min()
    end_date = all_assets.index.max()
    #date_range = pd.date_range(start=start_date, end=end_date, tz='US/Eastern')
    date_range = pd.date_range(start=start_date, end=end_date)
   
    len_tickers = len(tickers)
    for i in range(len_tickers):
        frame = all_assets.loc[all_assets.Symbol == tickers[i]].copy()
        frame = frame.reindex(date_range, fill_value=pd.NaT)
        frame = frame.asfreq('D')
        multi_asset.append(frame)

    return multi_asset

print('multi_asset_list function defined')

multi_asset_list function defined


In [92]:
# Generate our indicators for Buy Signals for each of our assets
# df: contains one company (symbol) of data
def add_indicators(df):
    # Sanity check for our indicators to work
    if len(df) < 250:
        return None
    
    df['NextOpen'] = df.Open.shift(-1)
    df['NextDate'] = df.index
    df['NextDate'] = df.NextDate.shift(-1)
    df['MA200'] = df['Close'].rolling(window=200).mean()
    
    df['Price_change'] = df['Close'].pct_change()
    df['Upmove']   = df['Price_change'].apply(lambda x: x if x > 0 else 0)
    df['Downmove'] = df['Price_change'].apply(lambda x: abs(x) if x < 0 else 0)
    df['Avg_up']   = df['Upmove'].ewm(span=19).mean()
    df['Avg_down']   = df['Downmove'].ewm(span=19).mean()
    
    df = df.dropna().copy()
    df['RS'] = df['Avg_up'] / df['Avg_down']
    df['RSI'] = df['RS'].apply(lambda x: 100 - (100 / (x + 1)))

    # df.loc[(df['Close'] > df['MA200']) & (df['RSI'] < 30), 'Buy_signal' ] = 'Yes'
    # df.loc[(df['Close'] <= df['MA200']) | (df['RSI'] >= 30), 'Buy_signal' ] = 'No'

    return df
    
print('add_indicators function defined....')

add_indicators function defined....


In [93]:
def add_buy_signals(df):
    buy_filter = (df['Close'] > df['MA200']) & (df['RSI'] < 30)
    df.loc[buy_filter, 'Buy_signal' ] = 'Yes'
    df.loc[(df['Close'] <= df['MA200']) | (df['RSI'] >= 30), 'Buy_signal' ] = 'No'


    return df

In [94]:
def add_sell_signals(df):
    
    pass

In [95]:
def generate_orders(df):
    pass

In [96]:
def order_simuator(df):
    max_hold_days = 10    # hold position for a maximum of 10 days
    stop_loss_percentage = -0.03
    target_percentage = 0.1
    # Order actions : Enter, Skip, Close, Stop, Target (a buy signal will be skipped if already in position)
    df['Order'] = ''

    in_position_start_index = None
    for index, row in df.iterrows():
        # Check for Enter position conditions
        if row.Buy_signal == 'Yes' and in_position_start_index is None:
            df.at[index, 'Order'] = 'Enter'
            in_position_start_index = index
        elif row.Buy_signal == 'Yes' and in_position_start_index is not None:
            df.at[index, 'Order'] = 'Skip'
        
        # Check for exit position conditions
        if in_position_start_index is not None:
            if (index - in_position_start_index).days >= max_hold_days:
                df.at[index, 'Order'] = 'Close'
                in_position_start_index = None
            elif index > in_position_start_index: 
                buy_price = df.at[in_position_start_index, 'Close']
                current_price = row['Close']
                percent_change = (current_price - buy_price) / buy_price
                if percent_change < stop_loss_percentage:
                    df.at[index, 'Order'] = 'Stop'
                    in_position_start_index = None
                if percent_change > target_percentage:
                    df.at[index, 'Order'] = 'Target'
                    in_position_start_index = None

    return df

print('Order simulator function defined....')

Order simulator function defined....


In [97]:
def extract_trades(df):
    # Loop through frame order information
    ## Grab Enter row to add to our trade dict
    ## Grab Sell row (Close, Target, Stop) to add to our ENTER row
    ## append to our trades
    ## Convert to a DataFrame
    # trade = {Id:999, Trade_seq: Tade_seq, Symbol: Ticker, Buy_dt: Date, Buy_Price: Price, Sell_dt: Date, Sell_Price: Price, Sell_Trigger: Close }
    # Id = symbol_Sort
    
    trades = []
    open_position = False
    trade_sequence = 1
    for index, row in df.iterrows():
        if not open_position: # Get the next Entry row
            if row.Order == 'Enter':
                trade_seq = trade_sequence
                symbol = row.Symbol
                buy_dt = row.NextDate
                buy_price = row.NextOpen
                open_position = True
        else: # Get the next Sell row (Close, Target, Stop)
            if row.Order in ['Close','Target','Stop']:
                sell_dt = row.NextDate
                sell_price = row.NextOpen
                sell_type = row.Order
                days_in_trade = (sell_dt - buy_dt).days
                profit = sell_price - buy_price
                percent_return = profit / buy_price
                trade = {'Id':f'{symbol}-{trade_seq}','Trade_seq':trade_seq,'Symbol':symbol,\
                         'Buy_dt':buy_dt,'Buy_Price':buy_price,\
                         'Sell_dt':sell_dt,'Sell_Price':sell_price,'Sell_type':sell_type,\
                         'Duration':days_in_trade,'profit':profit,'return':percent_return\
                        }
                trades.append(trade)
                open_position = False
                trade_sequence += 1
    trades_df = pd.DataFrame(trades)
    
    return trades_df

print('Extract Trades function defined....')  

Extract Trades function defined....


RUN SIMULATION FROM THIS POINT

In [98]:
# Grab symbols and get prices from yahoo
tickers = grab_symbols()
test_length = len(tickers)
all_assets = yahoo_prices(tickers[0:test_length], '2023-01-31')

[*********************100%***********************]  502 of 502 completed


In [ ]:
multi_assets = multi_asset_list(all_assets)
multi_assets

In [108]:
# add indicators and buy signals, generate orders and store detailed results by date and symbol in an easily accessible list
len_tickers = len(tickers[0:test_length])
for i in range(len_tickers):
    #frame = all_assets.loc[all_assets.Symbol == tickers[i]].copy()
    frame = multi_assets[i]
    frame = add_indicators(frame)
    frame = add_buy_signals(frame)
    if frame is not None:
        frame = order_simuator(frame)
        #multi_assets.append(frame)
        multi_assets[i] = frame

DataError: No numeric types to aggregate

In [13]:
all_trades = None
for i in range(len(multi_assets)):
    frame = multi_assets[i]
    trades = extract_trades(frame)
    if all_trades is None:
        all_trades = trades
    else:    
        all_trades = pd.concat([all_trades, trades], axis=0, ignore_index=True)


wining_trades = len(all_trades.loc[all_trades.profit > 0])
losing_trades = len(all_trades.loc[all_trades.profit < 0])
target_trades = len(all_trades.loc[all_trades.Sell_type == 'Target'])
stop_trades   = len(all_trades.loc[all_trades.Sell_type == 'Stop'])
close_trades  = len(all_trades.loc[all_trades.Sell_type == 'Close'])
print(f'Wining trades: {wining_trades}')
print(f'Losing trades: {losing_trades}')
print(f'Targets: {target_trades}, Stops: {stop_trades}, Closes: {close_trades}')
print(f'Total profit : ${all_trades.profit.sum():0.2f}')
all_trades

Wining trades: 1345
Losing trades: 1292
Targets: 134, Stops: 881, Closes: 1624
Total profit : $1741.27


,Id,Trade_seq,Symbol,Buy_dt,Buy_Price,Sell_dt,Sell_Price,Sell_type,Duration,profit,return
0,MMM-1,1,MMM,2021-01-12 00:00:00-05:00,165.899994,2021-01-22 00:00:00-05:00,170.199997,Close,10,4.300003,0.025919
1,MMM-2,2,MMM,2021-06-17 00:00:00-04:00,196.800003,2021-06-29 00:00:00-04:00,196.199997,Close,12,-0.600006,-0.003049
2,AOS-1,1,AOS,2021-09-15 00:00:00-04:00,68.139999,2021-09-20 00:00:00-04:00,64.029999,Stop,5,-4.110001,-0.060317
3,AOS-2,2,AOS,2022-01-21 00:00:00-05:00,78.230003,2022-01-26 00:00:00-05:00,76.610001,Stop,5,-1.620003,-0.020708
4,AOS-3,3,AOS,2022-01-27 00:00:00-05:00,77.010002,2022-02-08 00:00:00-05:00,74.040001,Close,12,-2.970001,-0.038566
...,...,...,...,...,...,...,...,...,...,...,...
2634,ZTS-1,1,ZTS,2019-11-11 00:00:00-05:00,117.290001,2019-11-19 00:00:00-05:00,119.010002,Close,8,1.720001,0.014665
2635,ZTS-2,2,ZTS,2021-01-28 00:00:00-05:00,154.300003,2021-02-09 00:00:00-05:00,159.809998,Close,12,5.509995,0.035710
2636,ZTS-3,3,ZTS,2021-09-29 00:00:00-04:00,194.710007,2021-10-11 00:00:00-04:00,197.559998,Close,12,2.849991,0.014637
2637,ZTS-4,4,ZTS,2022-01-06 00:00:00-05:00,215.479996,2022-01-14 00:00:00-05:00,206.940002,Stop,8,-8.539993,-0.039632


In [ ]:
tickers_list = grab_symbols()
tickers_list

In [5]:
data = yahoo_prices(tickers_list, '2023-01-31')
data

[*********************100%***********************]  502 of 502 completed


,Symbol,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,,
2018-01-31 00:00:00-05:00,A,70.709076,73.430000,74.389999,73.239998,73.769997,2032800.0
2018-01-31 00:00:00-05:00,AAL,52.943867,54.320000,54.709999,53.000000,53.080002,5962900.0
2018-01-31 00:00:00-05:00,AAP,109.981354,116.989998,120.470001,115.830002,119.860001,1235700.0
2018-01-31 00:00:00-05:00,AAPL,39.741600,41.857498,42.110001,41.625000,41.717499,129915600.0
2018-01-31 00:00:00-05:00,ABBV,88.762878,112.220001,116.500000,111.290001,116.209999,15744800.0
...,...,...,...,...,...,...,...
2023-01-30 00:00:00-05:00,YUM,129.039993,129.039993,129.309998,127.900002,127.959999,1457000.0
2023-01-30 00:00:00-05:00,ZBH,125.830002,125.830002,126.110001,125.180000,125.410004,2090700.0
2023-01-30 00:00:00-05:00,ZBRA,309.029999,309.029999,315.299988,308.750000,313.140015,240900.0


In [22]:
data.groupby("Symbol").count()

,Adj Close,Close,High,Low,Open,Volume
Symbol,,,,,,
A,1258,1258,1258,1258,1258,1258
AAL,1258,1258,1258,1258,1258,1258
AAP,1258,1258,1258,1258,1258,1258
AAPL,1258,1258,1258,1258,1258,1258
ABBV,1258,1258,1258,1258,1258,1258
...,...,...,...,...,...,...
YUM,1258,1258,1258,1258,1258,1258
ZBH,1258,1258,1258,1258,1258,1258
ZBRA,1258,1258,1258,1258,1258,1258


In [9]:
lines = data.groupby("Symbol").count()
lines.loc[lines.Volume != 1258]

,Adj Close,Close,High,Low,Open,Volume
Symbol,,,,,,
CARR,722,722,722,722,722,722
CDAY,1199,1199,1199,1199,1199,1199
CEG,259,259,259,259,259,259
CTVA,928,928,928,928,928,928
DOW,974,974,974,974,974,974
FOX,979,979,979,979,979,979
FOXA,980,980,980,980,980,980
GEHC,30,30,30,30,30,30
MRNA,1043,1043,1043,1043,1043,1043


In [26]:
# 2018-01-31
data.loc[data.Symbol == 'CARR'].index.min(), data.loc[data.Symbol == 'ZTS'].index.min()

(Timestamp('2020-03-19 00:00:00-0400', tz='America/New_York'),
 Timestamp('2018-01-31 00:00:00-0500', tz='America/New_York'))

In [27]:
data.loc[data.Symbol == 'CARR'].index.max(), data.loc[data.Symbol == 'ZTS'].index.max()

(Timestamp('2023-01-30 00:00:00-0500', tz='America/New_York'),
 Timestamp('2023-01-30 00:00:00-0500', tz='America/New_York'))

In [70]:
zts = data.loc[data.Symbol == 'ZTS'].copy()
print(f'Min date: {zts.index.min()}, number of dates: {len(zts)}')
zts = zts.asfreq('D')
print(f'Min date: {zts.index.min()}, number of dates: {len(zts)}')
#zts[zts.Close.isnull()]
zts


Min date: 2018-01-31 00:00:00-05:00, number of dates: 1258
Min date: 2018-01-31 00:00:00-05:00, number of dates: 1826


,Symbol,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,,
2018-01-31 00:00:00-05:00,ZTS,74.349388,76.730003,78.769997,76.540001,78.489998,4136400.0
2018-02-01 00:00:00-05:00,ZTS,75.405548,77.820000,78.269997,76.690002,76.839996,2981800.0
2018-02-02 00:00:00-05:00,ZTS,74.397842,76.779999,78.120003,76.730003,77.529999,2595200.0
2018-02-03 00:00:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-02-04 00:00:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2023-01-26 00:00:00-05:00,ZTS,168.240005,168.240005,168.300003,166.119995,166.869995,1530700.0
2023-01-27 00:00:00-05:00,ZTS,165.179993,165.179993,167.690002,164.500000,166.979996,1831700.0
2023-01-28 00:00:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
start_date = '2018-01-31'
end_date = '2023-01-30'
date_range = pd.date_range(start=start_date, end=end_date, tz='US/Eastern')

In [69]:
carr = data.loc[data.Symbol == 'CARR'].copy()
print(f'Min date: {carr.index.min()}, number of dates: {len(carr)}')
carr = carr.reindex(date_range, fill_value=pd.NaT)
carr = carr.asfreq('D')
print(f'Min date: {carr.index.min()}, number of dates: {len(carr)}')
carr

Min date: 2020-03-19 00:00:00-04:00, number of dates: 722
Min date: 2018-01-31 00:00:00-05:00, number of dates: 1826


,Symbol,Adj Close,Close,High,Low,Open,Volume
2018-01-31 00:00:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-02-01 00:00:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-02-02 00:00:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-02-03 00:00:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-02-04 00:00:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2023-01-26 00:00:00-05:00,CARR,43.650002,43.650002,44.0,43.169998,43.790001,2460100.0
2023-01-27 00:00:00-05:00,CARR,43.619999,43.619999,43.990002,43.380001,43.490002,1868900.0
2023-01-28 00:00:00-05:00,NaT,NaT,NaT,NaT,NaT,NaT,NaT
2023-01-29 00:00:00-05:00,NaT,NaT,NaT,NaT,NaT,NaT,NaT


In [66]:
carr

,Symbol,Adj Close,Close,High,Low,Open,Volume
Date,,,,,,,
2020-03-19 00:00:00-04:00,CARR,11.573104,12.000000,12.000000,12.000000,12.000000,100.0
2020-03-20 00:00:00-04:00,CARR,12.296424,12.750000,13.000000,12.000000,13.000000,2300.0
2020-03-23 00:00:00-04:00,CARR,11.129468,11.540000,12.750000,11.500000,12.750000,18700.0
2020-03-24 00:00:00-04:00,CARR,11.283776,11.700000,12.900000,11.500000,12.900000,96100.0
2020-03-25 00:00:00-04:00,CARR,12.971521,13.450000,13.450000,11.800000,11.800000,19200.0
...,...,...,...,...,...,...,...
2023-01-24 00:00:00-05:00,CARR,43.700001,43.700001,44.029999,42.860001,43.330002,4191600.0
2023-01-25 00:00:00-05:00,CARR,43.509998,43.509998,43.605000,42.799999,43.189999,2911700.0
2023-01-26 00:00:00-05:00,CARR,43.650002,43.650002,44.000000,43.169998,43.790001,2460100.0
